In [ ]:
import cv2
import numpy as np
import math

# 1. Initialize Webcam
cap = cv2.VideoCapture(0)

while True:
    try:
        # Capture frame-by-frame
        ret, frame = cap.read()
        if not ret: break
        
        frame = cv2.flip(frame, 1) # Mirror image for easier use
        kernel = np.ones((3, 3), np.uint8)
        
        # 2. Define Region of Interest (ROI)
        # We only look for the hand inside this square box
        roi = frame[100:400, 100:400]
        cv2.rectangle(frame, (100, 100), (400, 400), (0, 255, 0), 2)    
        
        # 3. Color Space Conversion (BGR to HSV)
        # Skin color is more consistent in HSV space
        hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
        
        # Skin color range (adjust these if your lighting is different)
        lower_skin = np.array([0, 20, 40], dtype=np.uint8)
        upper_skin = np.array([30, 150, 255], dtype=np.uint8)

        # Extract skin pixels
        mask = cv2.inRange(hsv, lower_skin, upper_skin)
        
        # 4. Morphological Math (Closing the gaps in the hand)
        mask = cv2.dilate(mask, kernel, iterations=4)
        mask = cv2.GaussianBlur(mask, (5, 5), 100) 
        
        # 5. Find Contours (The shape of the hand)
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        # Find the largest contour (assuming it's the hand)
        cnt = max(contours, key=lambda x: cv2.contourArea(x))
        
        # 6. Mathematical Hull and Area Ratio
        # areahull is the area of the "rubber band"
        # areacnt is the actual area of the hand pixels
        hull = cv2.convexHull(cnt)
        areahull = cv2.contourArea(hull)
        areacnt = cv2.contourArea(cnt)
        
        # This ratio helps identify a fist (0) vs a finger (1)
        arearatio = ((areahull - areacnt) / areacnt) * 100

        # 7. Convexity Defects (Finding the 'valleys' between fingers)
        hull_indices = cv2.convexHull(cnt, returnPoints=False)
        defects = cv2.convexityDefects(cnt, hull_indices)
        
        # Variable to count the finger gaps
        defects_count = 0 
        
        # 8. The Cosine Rule Logic
        for i in range(defects.shape[0]):
            s, e, f, d = defects[i, 0]
            start = tuple(cnt[s][0])
            end = tuple(cnt[e][0])
            far = tuple(cnt[f][0])
            
            # Use Euclidean Distance to find side lengths of the triangle
            a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
            b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
            c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
            
            # MATH: Law of Cosines to find the angle
            # angle = arccos((b^2 + c^2 - a^2) / 2bc)
            angle = math.acos((b**2 + c**2 - a**2) / (2 * b * c)) * 57
            
            # If angle < 90, it's a finger gap. 'd' is the depth of the valley
            if angle <= 90 and d > 3000:
                defects_count += 1
                cv2.circle(roi, far, 3, [255, 0, 0], -1) # Draw blue dots in gaps
            
            # Draw lines connecting the hull points
            cv2.line(roi, start, end, [0, 255, 0], 2)
            
        # 9. Final Decision Logic
        total_fingers = defects_count + 1

        if total_fingers == 1:
            if arearatio < 12:
                display_text = "0 (Fist)"
            elif arearatio < 17.5:
                display_text = "Best Guess: 1"
            else:
                display_text = "1 FInger"
        elif total_fingers == 2:
            display_text = "2 FIngers"
        elif total_fingers == 3:
            display_text = "3 FIngers"
        elif total_fingers == 4:
            display_text = "4 FIngers"
        elif total_fingers == 5:
            display_text = "5 FIngers"
        elif total_fingers == 6:
            display_text = "Reposition Hand"
        else:
            display_text = "Counting..."

        # 10. Output the results
        cv2.putText(frame, display_text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.imshow('Math-Based Gesture Recognition', frame)
        cv2.imshow('Binary Mask', mask) # See what the computer sees
        
    except Exception as e:
        # If no hand is found, the 'max' contour function fails
        cv2.putText(frame, "Put hand in box", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        cv2.imshow('Math-Based Gesture Recognition', frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()